<a href="https://colab.research.google.com/github/linked0/deep-rl/blob/master/discretization/Discretization_my.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 정리
- 이 코드는 openai gym 깃헙 사이트에서 gym 폴더만을 포함시켜서 테스트함.
- 로컬 피씨에서 jupyter notebook으로 실행시켜야 동작 UI를 확인할 수 있음.
- gym/envs/robotics 폴더는 사이즈때문에 지움.

# Discretization
---
In this notebook, you will deal with continuous state and action spaces by discretizing them. This will enable you to apply reinforcement learning algorithms that are only designed to work with discrete spaces.

### 1. Import Necessary Packages

In [1]:
import sys
import gym
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# Set plotting options
%matplotlib inline
plt.style.use('ggplot')
np.set_printoptions(precision=3, linewidth=120)


### 2. Specify the Environment, and Explore the State and Action Spaces
We'll use [OpenAI Gym](https://gym.openai.com/) environments to test and develop our algorithms. These simulate a variety of classic as well as contemporary reinforcement learning tasks. Let's use an environment that has a continuous state space, but a discrete action space.

In [2]:
# Create an environment and set random seed
env = gym.make('MountainCar-v0')
env.seed(505);

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
low:[-1.2  -0.07], high:[ 0.6   0.07] in Box


Run the next code cell to watch a random agent.

In [3]:
# Install Chainer, ChainerRL and CuPy!
import matplotlib
from IPython import display
%matplotlib inline

In [4]:
state = env.reset()
score = 0
for t in range(200):
  action = env.action_space.sample()
  env.render()
  state, reward, done, _ = env.step(action)
  score += reward
  if done:
    break
print('Final score:', score)
env.close()

Final score: -200.0


In this notebook, you will train an agent to perform much better! For now, we can explore the state and action spaces, as well as sample them.

In [5]:
# Explore state (observation) space
print('State space:', env.observation_space)
print('- low:', env.observation_space.low)
print('- high:', env.observation_space.high)

State space: Box(2,)
- low: [-1.2  -0.07]
- high: [ 0.6   0.07]


In [6]:
# Generate some samples from the state space
print("State space samples")
print(np.array([env.observation_space.sample() for i in range(10)]))

State space samples
[[-0.622 -0.039]
 [-0.946 -0.056]
 [ 0.571 -0.034]
 [-0.233 -0.007]
 [-1.021 -0.021]
 [-0.355  0.048]
 [ 0.428 -0.065]
 [-0.285 -0.047]
 [ 0.202  0.051]
 [-0.459 -0.05 ]]


In [7]:
# Explore the actio space
print('Action space:', env.action_space)

# Generate some samples from the action space
print('Action space samples:')
print(np.array([env.action_space.sample() for i in range(10)]))

Action space: Discrete(3)
Action space samples:
[1 1 1 2 2 2 0 1 2 1]


### 3. Discetize the State Space with a Uniform Grid
We will discretize the space using a uniformly-spaced grid. Implement the following function to create such a grid, give the lower bounds(low), upper bounds(high), and number of desired bins along each dimension. It should return the split points for each demension, which will be 1 less than the number of bins.

For instance, if `(low = [-1.0, -0.5]`, `high = [1.0, 5.0]`, and `bins = (10, 10)`, then your function should return the following list of 2 Numpy arrays:
```
[array([-0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8]),
array([-4.0, -3.0, -2.0. -1.0, 0.0, 1.0, 2.0, 3.0, 4.0])]
```
Nothe that the ends of `low` and `high` are **not** included in these split points. It is assumed that any value below the lowest split point maps index `0` and any value above the highest split point maps to index `n-1`, where `n` is the number of bins along that dimension.

In [14]:
def create_uniform_grid(low, high, bins=(10,10)):
  """ Define a uniformly-spaced grid that can be used to discretize a space.
  Parameters
  ----------
  low: array_like
      Lower bounds for each dimension of the continuous space.
  high: array_like
      Upper bounds for each dimension of the continuous space.
  bins: tuple
      Number of bins along each corresponding dimension.
  Returns
  -------
  grid: list of array_like
      A list of arrays containing split points for each dimension.
  """
  grid = [np.linspace(val[0], val[1], val[2] + 1)[1:-1] for val in zip(low, high, bins)]
  return grid

low = [-1.0, -5.0]
high = [1.0, 5.0]
create_uniform_grid(low, high) # [test]

[array([-0.8, -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8]),
 array([-4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.])]